In [2]:
# README!!!!!!!!!!!!!!!!!!!!!!!!
# This notebook is a simulation of how the p300 works
# To activate simulation press the >>  button and wait a few seconds. 
# [ >> that is the 4th button from the right in the upper line ('Restart the kernel and run all cells' button)]
# More explenation, in addition to the actual simulation, will be given at bottom of notebook
# README!!!!!!!!!!!!!!!!!!!!!!!!

from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import ipywidgets as widgets
from ipywidgets.widgets import Label, FloatProgress, FloatSlider, Button
from ipywidgets.widgets import Layout, HBox, VBox
from IPython.display import display
import numpy as np
import time
import threading
import logging
%matplotlib ipympl

#flag to control loop
flag = True
flag2 = True # todo change the flag name to something more clear
is_searching_col = None
chosen_char = 'A'
chosen_char_num = 0
last_was_found_in_row_col = -1
found_a_row_col = 0
# set up graph data
x = np.linspace(0, 10, 80)
#dx = x[1] - x[0]
dx = 0.125 # graph point created every 0.125 seconds
y = x*0

# Buttons GUI element

# change button's row and col into color
def reset_all_colors():
    for i in range(0,36):
            buttons[i].style.button_color='black'
        
# get number 0-11 and search said row/col for chosen char
def search_row_col(number):
    global found_a_row_col
    found_in_number = False
    global is_searching_col
    if (number <6):# it's of first 6 numbers so search col
        for i in range (0,6):
            current_search_index = number*6+i
            if (current_search_index == chosen_char_num):
                is_searching_col = False # found the right column so will now search row
                found_in_number = True
                found_a_row_col = 3
            buttons[current_search_index].style.button_color='gray'#to show it's been searched
    else:# it's of last 6 numbers so search row
        for i in range (0,6):
            current_search_index = number%6+i*6
            if (current_search_index == chosen_char_num):
                is_searching_col = True# found the right row so will now search column
                found_in_number = True
                found_a_row_col = 3
            buttons[current_search_index].style.button_color='gray'# showing it's been searched
    return found_in_number
        
    
# when button clicked function
def btn_clicked(b):
    global found_a_row_col
    found_a_row_col = 0
    global chosen_char
    chosen_char = b.description
    global chosen_char_num
    buttons[chosen_char_num].style.button_color = 'black'
    chosen_char_num = (ord(b.description)-ord('A'))
    if (chosen_char_num < 0):
        chosen_char_num = chosen_char_num + 43
    print("button: {}, char no. {}".format(chosen_char,chosen_char_num))
    # set flag to True
    global flag
    flag = True

# 36 char selection buttons
btn_layout = widgets.Layout(width = "auto", height = "auto")
buttons = []
for i in range (0,36):
    buttons.append(widgets.Button(description="{}".format(chr(65+i-43*int(i/26))), layout=btn_layout, style=dict(
        font_weight='bold',
        text_color='white',
        button_color='black')))
    buttons[i].on_click(btn_clicked)
box = widgets.GridBox(buttons, layout = widgets.Layout(
    grid_template_columns = "50px 50px 50px 50px 50px 50px",
    grid_template_rows = "50px 50px 50px 50px 50px 50px",
    ))

#loopfunction
def work():
    global flag
    global flag2
    global found_a_row_col
    found_a_row_col = 0
    global is_searching_col
    global x
    global y
    while flag2:# total updating both graph and grid
        if (flag):# grid part of updating
            # unhighlight last highlighted row/col
            reset_all_colors()
            if (is_searching_col == None): # if not searching specificly in col/row then search in random one of them
                random_row_col = np.random.randint(12)
                if (search_row_col(random_row_col)): # if chosen number is in said row/col
                    last_was_found_in_row_col = random_row_col # save said row/col
            elif (is_searching_col == False): # if not searching in col then search in rows
                random_row_col = np.random.randint(6)+6 # numbers 6-11 are rows
                if (search_row_col(random_row_col)): # if chosen number is in said row
                    #logging.warning("number was found in col no. {}, and row no. {}, which is number no. {}".format(last_was_found_in_row_col,random_row_col%6,last_was_found_in_row_col*6+random_row_col%6))
                    is_searching_col = None
                    flag = False
            else:# if searching in col - meaning is_searching_col == True
                random_row_col = np.random.randint(6) # numbers 0-5 are cols
                if (search_row_col(random_row_col)): # if chosen number is in said col
                    #logging.warning("number was found in col no. {}, and row no. {}, which is number no. {}".format(random_row_col,last_was_found_in_row_col%6,random_row_col*6+last_was_found_in_row_col%6))
                    is_searching_col = None
                    flag = False

        # update graph data
        x = np.delete(x,0)
        y = np.delete(y,0)
        x = np.append(x,x[-1]+dx)
        noise = -1+2*np.random.rand()
        y = np.append(y, 0+found_a_row_col + noise)
        
        # control cycle speed
        time.sleep(0.125)  #todo instead of 0.125 make it a variable with informative name
        found_a_row_col = 0 # making sure each p300 spike is only 1 point long


# set flag to True
flag = True
flag2 = True
# create thread to run work function
thread = threading.Thread(target = work)
# start thread
thread.start()

# graph making part
def animate(i):
    plt.cla()
    plt.plot(x,y)
    plt.xlabel('Time (s)')
    plt.ylabel('Potential (µV)')
    plt.title('P300 signals graph')
out = widgets.Output()
with out:
    ani = FuncAnimation(plt.gcf(), animate, interval = 250, cache_frame_data=False)
    plt.tight_layout()
    plt.show()

# stop/continue button creation
b_stop_continue = Button(
    description = 'stop',
    icons = 'stop',
    button_style = 'warning',
    layout = Layout(width = '100px')
)
def stop_continue_click(b):
    global flag
    global flag2
    if (flag2):
        flag = False
        flag2 = False
        b.description = 'continue'
    else:
        flag = True
        flag2 = True
        thread = threading.Thread(target = work)
        thread.start()
        b.description = 'stop'
b_stop_continue.on_click(stop_continue_click)

# Explenation text creation
explenation_widget = widgets.HTML(
    layout=Layout(width='400px', margin = '25px 25px 25px 25px'),
    value='<b><p style="text-align:center">P300 simulator explenation:</p></b><p>The following is a simulation that shows how using p300-based brain computer interface (BCI) looks like.</p><p>The 6x6 buttons grid simulate the interface, by clicking on a button you simulate the user concentrating on said button.</p><p>The interface will then start flashing rows and collums in order to see which button you meant, and when the chosen button collumn/row is flashed.</p><p>After approximately 300 miliseconds - the graph, which simulates brain signal detection of the p300 signal, will show a bump that the brain interface system uses to understand which button the simulated user supposedly looked at.</p>',
)

# Full interactive part of simulator
simulator_plus_stop_btn = HBox(
    children = (box, b_stop_continue)
)

# Adding graph to UI
simulator_plus_graph_box = VBox(
    layout=Layout(width='675px', margin = '25px 25px 25px 25px',display='flex', align_items='center'),
    children = (simulator_plus_stop_btn,out)
)

# Putting everything into one app
app = HBox(
    layout=Layout(border='solid 2px black', width='80%'),
    children = (explenation_widget, simulator_plus_graph_box)
)
# display app
display(app)